# TECH CHALLENGE
Tech Challenge é o projeto que engloba os conhecimentos obtidos em todas as disciplinas presentes na fase. Esta é uma atividade que, em princípio, deve ser desenvolvida em grupo. É importante atentar-se ao prazo de entrega, uma vez que essa atividade é obrigatória, valendo 90% da nota de todas as disciplinas da fase.

**O PROBLEMA**

No Tech Challenge desta fase, você precisa executar o fine-tuning de um foundation model (Llama, BERT, MISTRAL etc.), utilizando o dataset "The AmazonTitles-1.3MM". O modelo treinado deverá:


*   Receber perguntas com um contexto obtido por meio do arquivo json “trn.json”
*   A partir do prompt formado pela pergunta do usuário sobre o título do produto, o modelo deverá gerar uma resposta baseada na pergunta do usuário trazendo como resultado do aprendizado do fine-tuning os dados da sua descrição que está contido dentro do dataset.

### Fluxo de trabalho atualizado:
1. Escolha do Dataset:
Descrição: o The AmazonTitles-1.3MM consiste em consultas textuais reais de usuários e títulos associados de produtos relevantes encontrados na Amazon e suas descrições, medidos por ações implícitas ou explícitas dos usuários.

2. Preparação do Dataset:
Faça o download do dataset AmazonTitles-1.3MM e utilize o arquivo “trn.json”. Nele, você utilizará as colunas “title” e “content”, que contêm título e descrição respectivamente. Prepare os prompts para o fine-tuning garantindo que estejam organizados de maneira adequada para o treinamento do modelo escolhido. Limpe e pré-processe os dados conforme necessário para o modelo escolhido.

3. Chamada do Foundation Model
Importe o foundation model que será utilizado e faça um teste apresentando o resultado atual do modelo antes do treinamento (para que se obtenha uma base de análise após o fine-tuning), e então será possível avaliar a diferença do resultado gerado.

4. Execução do Fine-Tuning:
Execute o fine-tuning do foundation model selecionado (por exemplo,
BERT, GPT, Llama) utilizando o dataset preparado. Documente o processo de fine-tuning, incluindo os parâmetros utilizados e qualquer ajuste específico realizado no modelo.

5. Geração de Respostas:
Configure o modelo treinado para receber perguntas dos usuários. O modelo deverá gerar uma resposta baseada na pergunta do usuário e nos dados provenientes do fine-tuning, incluindo as fontes fornecidas. O que esperamos para o entregável?



*   Documento detalhando o processo de seleção e preparação do dataset.
*   Descrição do processo de fine-tuning do modelo, com detalhes dos parâmetros e ajustes utilizados. Código-fonte do processo de fine- tuning.
*   Um vídeo demonstrando o modelo treinado gerando respostas a partir de perguntas do usuário e utilizando o contexto obtido por meio treinamento com o fine-tuning.

In [1]:

#rodar esse a parte
# Montando volume compartilhado no google drive para salvar os arquivos localmente
# Isso nos permite reuso em diversos pontos desse notebook
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Instalação de Pacotes

In [2]:
#rodar esse a parte
# Instalando pacotes necessários.
# Decidimos utilizar o unsloth para acelerar o processo de fine-tuning, já que o dataset é muito grande
!pip install bert-tensorflow
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install xformers==0.0.27 "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 6.7 MB/s eta 0:00:00
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-2tzbko79/unsloth_1546d97fabd64bf197e91155cda10907
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-2tzbko79/unsloth_1546d97fabd64bf197e91155cda10907
  Resolved https://github.com/unslothai/unsloth.git to commit f1951c0f6d3e1f184af93e5d8f5eff6e7834e4b5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   

# Preparação do Dataset

Nos códigos abaixos:



*   Fazemos o download do dataset a partir do google drive
*   Extraímos o arquivo trn.json, que contém todos os dados que nós queremos
*   Processamos e limpamos o dataset, preparando para o fine-tuning



### Download do Dataset

In [3]:
!gdown 12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=cfec92b9-0c6b-424d-bea1-c9b874af97f2
To: /content/LF-Amazon-1.3M.raw.zip
100% 890M/890M [00:09<00:00, 93.3MB/s]


### Extração do Dataset

In [4]:
import zipfile
zip_path = '/content/LF-Amazon-1.3M.raw.zip'
extract_to = '/content/drive/MyDrive/FIAP/content/datasets'
zipfile.ZipFile(zip_path, 'r').extractall(extract_to)

#REALIZAR O CHUNK DO DATASET EM 50K CADA

In [5]:
import pandas as pd
import os
drive = '/content/drive/MyDrive/FIAP'
# Lê o arquivo JSON compactado em pedaços de 100.000 linhas, armazena os dados em um DataFrame.
datasets_path = f'{drive}/content/datasets/LF-Amazon-1.3M'
trn_filepath = f'{datasets_path}/trn.json.gz'

chunks_dir = f'{drive}/content/datasets/chunks/'

# Cria o diretório de chunks se não existir
os.makedirs(chunks_dir, exist_ok=True)

# Define o tamanho do chunk
chunk_size = 100000

chunks = pd.read_json(trn_filepath, lines=True, chunksize=chunk_size)

# Itera sobre cada pedaço e salva os chunks
for i, chunk_df in enumerate(chunks):
    # Define o caminho para o novo arquivo
    chunk_filepath = f'{chunks_dir}chunk_{i}.json.gz'

    # Salva o chunk em um arquivo JSON comprimido
    chunk_df.to_json(chunk_filepath, orient='records', lines=True, compression='gzip')

    print(f'Salvo: {chunk_filepath}')



Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_0.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_1.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_2.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_3.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_4.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_5.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_6.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_7.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_8.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_9.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_10.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_11.json.gz
Salvo: /content/drive/MyDrive/FIAP/content/datasets/chunks/chunk_12.json.gz
Salvo: /content/drive/

#CRIAR FUNÇÃO DE LIMPEZA

In [6]:
import re
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from wordcloud import STOPWORDS
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.filterwarnings('ignore')

def limpeza_texto(text):
    # regex para limpar o texto
    whitespace = re.compile(r"\s+")                            # encontrando espaços em branco
    user = re.compile(r"(?i)@[a-z0-9_]+")                      # encontrar menções de usuários, exemplo @usuario
    text = whitespace.sub(' ', text)                           # substitui espaços em branco por ' '
    text = user.sub('', text)                                  # remove todas as menções de usuário encontradas no texto
    text = re.sub(r"\[[^()]*\]","", text)                      # remove o conteúdo dentro de colchetes, incluindo os colchetes
    text = re.sub("\d+", "", text)                             # remove todos os dígitos numéricos do texto
    text = re.sub(r'[^\w\s]','',text)                          # remove todos os caracteres que não são palavras (letras e números) ou espaços em branco.
    text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "", text) # remove menções de usuário, hashtags e URLs.
    text = text.lower()                                        # texto para minusculo

    # removendo as stop words
    text = [word for word in text.split() if word not in list(STOPWORDS)]

    # word lemmatization
    sentence = []
    for word in text:
        lemmatizer = WordNetLemmatizer()
        sentence.append(lemmatizer.lemmatize(word,'v'))

    return ' '.join(sentence)

[nltk_data] Downloading package wordnet to /root/nltk_data...


#REALIZAR A LIMPEZA DOS CHUNKS

In [7]:
import json
chunks_dir_clear = f'{drive}/content/datasets/chunks_clear/'

# Cria o diretório de chunks_clear se não existir
os.makedirs(chunks_dir_clear, exist_ok=True)


#pipeline limpeza e chunck json limpo

for filename in os.listdir(chunks_dir):
    if filename.endswith('.json.gz'):
        # Caminho completo do arquivo de entrada
        file_path = os.path.join(chunks_dir, filename)

        # Lê o arquivo JSON em pedaços (chunks)
        json_reader = pd.read_json(file_path, lines=True, chunksize=50000)
        df_trn = next(json_reader)

        # Aplicar a limpeza
        df_trn = df_trn[['title', 'content']]
        df_trn['title'] = df_trn['title'].str.strip()
        df_trn['content'] = df_trn['content'].str.strip()
        df_trn = df_trn[df_trn['title'] != '']
        df_trn = df_trn[df_trn['content'] != '']
        df_trn = df_trn.dropna(subset=['title', 'content'])
        df_trn['title_clear'] = df_trn['title'].apply(limpeza_texto)
        df_trn['content_clear'] = df_trn['content'].apply(limpeza_texto)

        # Caminho para o arquivo limpo
        output_filename = f'cleaned_{filename}'
        output_file_path = os.path.join(chunks_dir_clear, output_filename)

        # Salva o dataset processado e limpo em um arquivo JSON
        df_trn.to_json(output_file_path, orient='records', lines=True, compression='gzip')

        print(f'Arquivo limpo salvo em: {output_file_path}')


Arquivo limpo salvo em: /content/drive/MyDrive/FIAP/content/datasets/chunks_clear/cleaned_chunk_0.json.gz
Arquivo limpo salvo em: /content/drive/MyDrive/FIAP/content/datasets/chunks_clear/cleaned_chunk_1.json.gz
Arquivo limpo salvo em: /content/drive/MyDrive/FIAP/content/datasets/chunks_clear/cleaned_chunk_2.json.gz
Arquivo limpo salvo em: /content/drive/MyDrive/FIAP/content/datasets/chunks_clear/cleaned_chunk_3.json.gz
Arquivo limpo salvo em: /content/drive/MyDrive/FIAP/content/datasets/chunks_clear/cleaned_chunk_4.json.gz
Arquivo limpo salvo em: /content/drive/MyDrive/FIAP/content/datasets/chunks_clear/cleaned_chunk_5.json.gz
Arquivo limpo salvo em: /content/drive/MyDrive/FIAP/content/datasets/chunks_clear/cleaned_chunk_6.json.gz
Arquivo limpo salvo em: /content/drive/MyDrive/FIAP/content/datasets/chunks_clear/cleaned_chunk_7.json.gz
Arquivo limpo salvo em: /content/drive/MyDrive/FIAP/content/datasets/chunks_clear/cleaned_chunk_8.json.gz
Arquivo limpo salvo em: /content/drive/MyDrive

#CRIAR/CARREGAR A BASE DO MODELO

In [8]:
#rodar esse a parte
#base modelo treino
import torch

#from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer

model_dir = f'{drive}/content/datasets/model/'

# Cria o diretório de model se não existir
os.makedirs(model_dir, exist_ok=True)

#df_trn  pegar cada arquivo cleaned e jogar aqui no df


max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Decidimos utilizar o modelo Llama
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.for_inference(model)


model = FastLanguageModel.get_peft_model(
    model,
    r = 8, #16,
    target_modules = ["q_proj", "k proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!


Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.9 patched 32 layers with 0 QKV layers, 32 O layers and 32 MLP layers.


#CARREGAR CHUNKS LIMPOS E SEPARAR EM TEST AND TRAIN NO FORMATO DO PROMPT

In [10]:
# pipeline para treino
from datasets import load_dataset, Dataset

for filename in os.listdir(chunks_dir_clear):
    if filename.endswith('.json.gz'):
        # Caminho completo do arquivo de entrada
        file_path = os.path.join(chunks_dir_clear, filename)

        # Lê o arquivo JSON em pedaços (chunks)
        json_reader = pd.read_json(file_path, lines=True, chunksize=100000)
        df_trn = next(json_reader)

        df_trn['input'] = df_trn.apply(
            lambda row: f"DESCRIBE THIS PRODUCT:\n[|Title|]{row['title_clear']}[|Title|]\n[|Description|]{row['content_clear']}[|Description|]",
            axis=1
        )


        instruction_list = ["DESCRIBE THIS PRODUCT."] * len(df_trn)

        input_list = df_trn['title_clear'].tolist()
        output_list = df_trn['content_clear'].tolist()

        data = {
            "instruction": instruction_list,
            "input": input_list,
            "output": output_list
        }

        prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        {}

        ### Input:
        {}

        ### Response:
        {}"""

        EOS_TOKEN = tokenizer.eos_token

        def formatting_prompts_func(data):
            instructions = data["instruction"]
            inputs = data["input"]
            outputs = data["output"]
            texts = []

            for instruction, input, output in zip(instructions, inputs, outputs):
                text = prompt.format(instruction, input, output) + EOS_TOKEN
                texts.append(text)
            return { "text": texts, }


        output_filename = f'model_{filename}'
        output_file_path = os.path.join(model_dir, output_filename)

        # Salva o dataset processado e limpo em um arquivo JSON
        #df = pd.DataFrame(data)
        #print(f'Arquivo limpo salvo em: {output_file_path}')
        #dataset = load_dataset("json", data_files=f"{drive}/datasets/model/model_{i}_formatted.json", split = "train")
        dataset = formatting_prompts_func(data)
        df = pd.DataFrame(dataset)


        df.to_json(output_file_path, orient='records', lines=True, compression='gzip')
        dataset = Dataset.from_pandas(df)

        # Divida o Dataset em conjuntos de treinamento e teste
        split_dataset = dataset.train_test_split(test_size=0.1, seed=42)

        # Defina os caminhos de saída
        output_filename_train = f'model_train_{filename}'
        output_file_path_train = os.path.join(model_dir, output_filename_train)

        output_filename_test = f'model_test_{filename}'
        output_file_path_test = os.path.join(model_dir, output_filename_test)

        # Salve o Dataset de treinamento em JSON comprimido
        train_dataset = split_dataset['train']
        train_df = train_dataset.to_pandas()
        train_df.to_json(output_file_path_train, orient='records', lines=True, compression='gzip')

        # Salve o Dataset de teste em JSON comprimido
        test_dataset = split_dataset['test']
        test_df = test_dataset.to_pandas()
        test_df.to_json(output_file_path_test, orient='records', lines=True, compression='gzip')



        print(f'Arquivo model salvo em: {output_file_path}')
        # with open(f'{model_dir}{output_file_path}.json', 'w') as f:
        #     json.dump(dataset.to_dict(), f, indent=4)




Arquivo model salvo em: /content/drive/MyDrive/FIAP/content/datasets/model/model_cleaned_chunk_0.json.gz
Arquivo model salvo em: /content/drive/MyDrive/FIAP/content/datasets/model/model_cleaned_chunk_1.json.gz
Arquivo model salvo em: /content/drive/MyDrive/FIAP/content/datasets/model/model_cleaned_chunk_2.json.gz
Arquivo model salvo em: /content/drive/MyDrive/FIAP/content/datasets/model/model_cleaned_chunk_3.json.gz
Arquivo model salvo em: /content/drive/MyDrive/FIAP/content/datasets/model/model_cleaned_chunk_4.json.gz
Arquivo model salvo em: /content/drive/MyDrive/FIAP/content/datasets/model/model_cleaned_chunk_5.json.gz
Arquivo model salvo em: /content/drive/MyDrive/FIAP/content/datasets/model/model_cleaned_chunk_6.json.gz
Arquivo model salvo em: /content/drive/MyDrive/FIAP/content/datasets/model/model_cleaned_chunk_7.json.gz
Arquivo model salvo em: /content/drive/MyDrive/FIAP/content/datasets/model/model_cleaned_chunk_8.json.gz
Arquivo model salvo em: /content/drive/MyDrive/FIAP/con

#CRIAR OS CHUNKS DE TREINO TOKENIZADO UTILIZANDO OS CHUNKS SPLITADOS EM TRAIN E TEST

In [11]:

import os
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import TrainingArguments
from trl import SFTTrainer

drive = '/content/drive/MyDrive/FIAP'
model_dir = f'{drive}/content/datasets/model/'

treino_dir = f'{drive}/content/datasets/treino/'

# Cria o diretório de treino se não existir
os.makedirs(treino_dir, exist_ok=True)

# Configurações do Trainer (ajuste conforme necessário)
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=60,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs"
)



all_files = os.listdir(model_dir)

# Filtra os arquivos que começam com 'model_' e terminam com '.json.gz'
filtered_files = [f for f in all_files if f.startswith('model_') and f.endswith('.json.gz')]

# Conta o número total de arquivos
total_files = len(filtered_files)

# Calcula o número de chunks (cada chunk tem 3 arquivos)
num_chunks = total_files // 3  # Divida por 3 porque cada chunk tem 3 arquivos
for i in range(num_chunks):
#for i in range(num_chunks):  # Ajuste `num_chunks` conforme o número de chunks disponíveis
    # Define paths
    train_file = f'{model_dir}model_train_cleaned_chunk_{i}.json.gz'
    test_file = f'{model_dir}model_test_cleaned_chunk_{i}.json.gz'



    # Carrega datasets
    train_df = pd.read_json(train_file, compression='gzip', lines=True)
    test_df = pd.read_json(test_file, compression='gzip', lines=True)

    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    print(train_dataset)

    # Instancia o Trainer
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        packing=False,
        dataset_num_proc=1,
        args=training_args
    )

    # Treine o modelo
    trainer.train()

    # Salve o modelo e o tokenizer
    model_save_path = f'{treino_dir}model_chunk_{i}/'
    os.makedirs(model_save_path, exist_ok=True)
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

    print(f'Modelo e tokenizer salvos em: {model_save_path}')

    # Se desejar, você pode adicionar uma etapa de limpeza aqui para liberar memória ou outros procedimentos necessários

print("Treinamento concluído para todos os chunks.")


Dataset({
    features: ['text'],
    num_rows: 34582
})


Map:   0%|          | 0/34582 [00:00<?, ? examples/s]

Map:   0%|          | 0/3843 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 34,582 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 19,660,800


Step,Training Loss
1,4.874200
2,4.901400
3,4.975900
4,4.891600
5,4.823100
6,4.830400
7,4.336700
8,4.220800
9,4.051000
10,4.090600


...

Map:   0%|          | 0/22496 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 22,496 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 19,660,800


Step,Training Loss
1,2.562800
2,2.111100
3,2.055900
4,1.915000
5,2.861300
6,3.285300
7,2.005300
8,2.744300
9,2.344200
10,2.960500


Modelo e tokenizer salvos em: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_22/
Treinamento concluído para todos os chunks.


#CONCATENAR OS CHUNKS TREINO E DEIXAR 1 MODEL APENAS PARA A INFERÊNCIA

In [14]:
import os
import json

drive = '/content/drive/MyDrive/FIAP'
treino_dir = f'{drive}/content/datasets/treino/'
full_dir = f'{drive}/content/datasets/treino_full/'


# Cria o diretório de treino se não existir
os.makedirs(full_dir, exist_ok=True)
# Defina os nomes dos arquivos JSON a serem mesclados
json_files = ['special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json']

def merge_json_files(json_paths):
    merged_data = {}
    for json_path in json_paths:
        with open(json_path, 'r') as file:
            data = json.load(file)
            merged_data = dict(merged_data, **data)  # Mesclar os dados, substituindo chaves se necessário
    return merged_data

def process_chunks(chunk_dirs):
    for chunk_dir in chunk_dirs:
        chunk_path = os.path.join(treino_dir, chunk_dir)
        if os.path.isdir(chunk_path):
            json_paths = [os.path.join(chunk_path, file) for file in json_files]
            if all(os.path.exists(path) for path in json_paths):
                # Mesclar arquivos JSON do chunk
                merged_data = merge_json_files(json_paths)

                # Crie o diretório de saída se não existir
                os.makedirs(full_dir, exist_ok=True)

                # Salvar arquivos mesclados com sufixo _complete no novo diretório
                for json_file in json_files:
                    complete_path = os.path.join(full_dir, f'{json_file.replace(".json", "_complete.json")}')
                    with open(complete_path, 'w') as file:
                        json.dump(merged_data, file, indent=4)

                print(f'Merged and saved: {chunk_path}')
            else:
                print(f'Missing one or more JSON files in {chunk_path}')

# Obtenha a lista de diretórios de chunks
chunk_dirs = [d for d in os.listdir(treino_dir) if os.path.isdir(os.path.join(treino_dir, d))]

# Processar todos os chunks
process_chunks(chunk_dirs)

print("Mesclagem concluída. Arquivos salvos em:", full_dir)


Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_0
Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_1
Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_2
Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_3
Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_4
Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_5
Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_6
Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_7
Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_8
Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_9
Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model_chunk_10
Merged and saved: /content/drive/MyDrive/FIAP/content/datasets/treino/model

In [ ]:
# eval_results = trainer.evaluate()
# print(f"Acurácia do modelo: {eval_results.get('eval_accuracy', 'Não disponível'):.2f}")

## Inferência com o Modelo Treinado

Realiza a inferência com o modelo treinado, gerando uma descrição do produto de entrada fornecido e exibindo a saída gerada.

In [ ]:
from transformers import TextStreamer

drive = '/content/drive/MyDrive/FIAP'
full_dir = f'{drive}/content/datasets/treino_full/'
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = full_dir,
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model)

inputs = tokenizer([
    prompt.format(
        "DESCRIBE THIS PRODUCT.",
        "Apple A Day: The Myths, Misconceptions and Truths About the Foods We Eat",
        "",
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)
# tokenizer.batch_decode(outputs)